# Details Of website


# Install All the Required Packages

In [1]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers

In [2]:
!pip -q install unstructured

In [3]:
!pip install tokenizers

In [4]:
!pip install xformers

In [5]:
!pip install pinecone-client==2.2.4

In [6]:
pip install -U langchain-community

#**2: Import All the Required Libraries**

In [7]:
# Dependency packages
!pip install numpy==1.24.4
!pip install nltk==3.9.1

In [8]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [9]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Pass the URLs and extract the data from these URLs

In [10]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'
]

In [11]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [12]:
data # Document Format

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko\n\nFollow\n\nPublished in\n\nGoPenAI\n\n15 min read\n\nJul 20, 2023\n\n--\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their ap

In [13]:
len(data)

4

# Split the Text into Chunks

In [14]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [15]:
text_chunks=text_splitter.split_documents(data)

In [16]:
len(text_chunks)

86

In [17]:
text_chunks

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhan

In [18]:
len(text_chunks)

86

In [19]:
text_chunks[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhanc

In [20]:
text_chunks[1]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Pretraining\nThe authors developed the Llama 2 model family starting from the pretraining methodology of Llama, which utilizes an optimized auto-regressive transformer. They implemented several modifications for improved performance, including enhanced data cleaning, updated data mixes, training on 40% more total tokens, and doubling the context length. They also incorporated grouped-query attention (GQA) to enhance the inference scalability for their larger models.\nPretraining Data')

In [21]:
text_chunks[2]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Pretraining Data\nThe authors utilized a novel mix of data from publicly accessible sources to train the Llama 2 models, excluding any data from Meta’s products or services. They made efforts to erase data from certain sites known for harboring large amounts of personal information about private individuals. They trained the models on 2 trillion tokens of data, believing this amount provided a beneficial performance-cost balance. They also up-sampled the most factual sources to boost knowledge and reduce instances of false information generation or “hallucinations”.\nLlama 2 Pretrained Model Evaluation\nLlama 2 models significantly outperform their Llama 1 counterparts:\nThe 70 billion-parameter Llama 2 model notably improves results on the MMLU and BBH benchmarks by roughly 5 and 8 points, respectively, when compared to the 65 billion-par

# Download the Hugging Face Embeddings

In [22]:
#embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings=HuggingFaceEmbeddings()

<ipython-input-22-71386b1e8255>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
<ipython-input-22-71386b1e8255>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingfa

In [23]:
embeddings
# Embedding model sentence-transformers/all-mpnet-base-v2

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [24]:
# Embedding size
query_result = embeddings.embed_query("How are you")
len(query_result)

768

In [25]:
query_result

[0.024816852062940598,
 0.04983457177877426,
 0.002963210456073284,
 -0.020912012085318565,
 0.01575615629553795,
 0.0008427133434452116,
 -0.03886730596423149,
 -0.0012208165135234594,
 0.017344709485769272,
 -0.01265059132128954,
 -0.0334111712872982,
 -0.021266302093863487,
 0.0026828490663319826,
 0.01360261719673872,
 0.0033968226052820683,
 -0.053298335522413254,
 -0.013983198441565037,
 -0.057532504200935364,
 -0.0442868135869503,
 -0.014237094670534134,
 -0.05664316937327385,
 0.011137329041957855,
 0.027455976232886314,
 -0.0027559702284634113,
 0.051405370235443115,
 -0.0021438507828861475,
 0.02927929349243641,
 -0.0008696287986822426,
 -0.011007800698280334,
 0.04820593446493149,
 -0.028723593801259995,
 0.02568708546459675,
 0.014495950192213058,
 -0.002313037170097232,
 1.6342876278940821e-06,
 0.06020195037126541,
 -0.026786038652062416,
 -0.03696710243821144,
 0.0686761885881424,
 -0.010282870382070541,
 0.020827023312449455,
 -0.08207922428846359,
 0.00636717677116394,

# Convert the Text Chunks into Embeddings and Create a Knowledge Base

In [26]:
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY','api key')
PINECONE_API_ENV= os.environ.get('PINECONE_API_ENV','env')

In [30]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)

In [29]:
index_name='llama'

In [ ]:
vectorstore=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
vectorstore=Pinecone.from_documents(text_chunks, embeddings, index_name=index_name)

# Create a Large Language Model (LLM) Wrapper

In [32]:
# For Loading the model
notebook_login()

In [33]:
# Model NAme
model = "meta-llama/Llama-2-7b-chat-hf"
# Model = "daryl149/llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model,
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

In [ ]:
# Initialize the pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

" and the Philosopher's Stone by J.K. Rowling.\nHarry Potter and the Philosopher's Stone is a fantasy novel written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter, who discovers that he is the chosen one to defeat the dark wizard, Lord Voldemort. Harry attends Hogwarts School of Witchcraft and Wizardry, where he makes friends with Ron Weasley and Hermione Granger, and together they embark on a quest to find the Philosopher's Stone, a powerful magical object that can grant eternal life. Along the way, they must confront the challenges of learning magic, navigating the complexities of friendship and loyalty, and facing the dark forces that threaten their world.\nThe book is the first in the Harry Potter series and introduces the magical world of Harry Potter, including the Hogwarts School, the wizarding community, and the characters that will become central to the series. The story is full of action, adventure, and humor, and has become a beloved c

# Initialize the Retrieval QA with Source Chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
query = "How good is Vicuna?"
# From Url

In [ ]:
docs = vectorstore.similarity_search(query, k=3)

In [ ]:
docs

[Document(page_content='Preliminary evaluations based on GPT-4, summarized in Figure 1, show that Vicuna achieves 90%* capability of Bard/ChatGPT.\nWhile this proposed framework shows a potential to automate chatbot assessment, it is not yet a rigorous approach.\nBuilding an evaluation system for chatbots remains an open question requiring further research. More details are provided in the evaluation section.\nFigure 1. Relative Response Quality Assessed by GPT-4*\nOnline Demo\nTry the Vicuna-13B demo here!\nOverview', metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}),
 Document(page_content='Preliminary evaluations based on GPT-4, summarized in Figure 1, show that Vicuna achieves 90%* capability of Bard/ChatGPT.\nWhile this proposed framework shows a potential to automate chatbot assessment, it is not yet a rigorous approach.\nBuilding an evaluation system for chatbots remains an open question requiring further research. More details are provided in the evaluation sect

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [ ]:
query = "What is is stableLM?"
qa.run(query)

' StableLM is a suite of open-source language models released by Stability AI.\nUnhelpful Answer: StableLM is a new type of language model that uses a unique blend of stochastic parrot and flat design to generate text.'

In [ ]:
# QA Format
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: what mpt 7b


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer:  MPT-7B is a 7-billion parameter language model developed by the LLM Foundry. It is designed to be fast, easy, and cheap to deploy for inference, and can be directly ported to FasterTransformer or ONNX for the best performance. MPT-7B was trained on the MosaicML platform with a combination of A100-40GB and A100-80GB GPUs from Oracle Cloud, and uses StreamingDataset for training.
Input Prompt: exit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
